In [20]:

from pandas import read_excel
import numpy as np
from queue import PriorityQueue
from collections import deque
import random

#상수 선언
LIMIT=20    # 엘리베이터의 탑승정원
OPEN_TIME=8    # 엘리베이터 문이 열리는 시간
CLOSE_TIME=11    # 엘리베이터 문이 닫히는 시간
CONGESTION=0.0001 # 엘리베이터의 혼잡도, 높아질수록 생성되는 승객이 많아진다.
 
    
    
    
    
#변수 선언
state=[0,0,0,0,0] # 엘리베이터의 상태, ex) 12명의 승객을 태우고 3층에서 7층으로 상승 중(1)이고, 12345초에 도착 예정이라면, state=[12, 1, 3, 7,12345] 
passenger=[] # 엘리베이터에 탄 승객
call_up=[] # 엘리베이터의 위층으로 가는 호출
call_down=[] # 엘리베이터의 아래층으로 가는 호출
passenger_up=[[] for i in range(0,10)] # 위층으로 가고 싶은 승객들 ex) (4,8,0) -> 4층에서 8층으로 가고 싶은 승객, 0초 기다림 
passenger_down=[[] for i in range(0,10)] # 아래층으로 가고 싶은 승객들



floor = [] #층


time_seat=[[0 for col in range(604800)] for row in range(10)] # 각 층의 시각 별 우선도
#초기값 설정
for i in range(604800):
    time_seat[0][i]+=100
    time_seat[1][i]+=50
    time_seat[2][i]+=100
    time_seat[3][i]+=50
    time_seat[4][i]+=50
    time_seat[5][i]+=50
    time_seat[6][i]+=100
    time_seat[7][i]+=50
    time_seat[8][i]+=100
    time_seat[9][i]+=50
    
time_seat2=[[0 for col in range(604800)] for row in range(10)] # 각 층의 시각 별 생성되는 승객 비율 (출발지점)
for i in range(604800):
    time_seat2[0][i]+=400
    time_seat2[1][i]+=50
    time_seat2[2][i]+=100
    time_seat2[3][i]+=50
    time_seat2[4][i]+=50
    time_seat2[5][i]+=50
    time_seat2[6][i]+=100
    time_seat2[7][i]+=50
    time_seat2[8][i]+=400
    time_seat2[9][i]+=50


time_seat3=[[0 for col in range(604800)] for row in range(10)] # 각 층의 시각 별 생성되는 승객 비율 (도착지점)
for i in range(604800):
    time_seat3[0][i]+=400
    time_seat3[1][i]+=50
    time_seat3[2][i]+=100
    time_seat3[3][i]+=50
    time_seat3[4][i]+=50
    time_seat3[5][i]+=50
    time_seat3[6][i]+=100
    time_seat3[7][i]+=50
    time_seat3[8][i]+=400
    time_seat3[9][i]+=50
    

# 엘리베이터 알고리즘 평가를 위한 변수들
sum_passenger=0 # 모든 승객 수의 합
sum_waiting=0 # 모든 승객들이 기다린 시간의 합
average_waiting=0 # 모든 승객들이 기다린 시간의 평균
std_waiting=0 # 이용자의 대기시간 표준편차
sum_distance=0 # 엘리베이터의 총 이동 거리
average_distance=0 # 엘리베이터의 총 이동 거리/옮긴 승객
            
 
            
            
# 시간별로 랜덤한 도착 층을 설정한다
def randArrival(time):
    sum_arrival=0    #도착 지점 승객 비율의 합
    accumulate=0
    rand=random.random() #도착 지점을 정하는 난수
    arrival=-1 # 도착 층
    
    for j in range(0, 10):
        sum_arrival+=time_seat3[j][i]
    
    #푸아송 분포를 통해 1초마다 탑승을 희망하는 승객을 생성한다.
    for j in range(0, 10):
        if(((accumulate/sum_arrival)<=rand)and(rand<(accumulate+time_seat3[j][i])/sum_arrival)):
            arrival=j
            #print(str(j+1)+"층")
        accumulate+=time_seat3[j][i]
    return arrival
        


def advance_call(now):
    global sum_distance
    global sum_passenger
    global sum_waiting
    global std_waiting
    state[2]=state[3] #도착
    
    #승객을 내린다.
    i=0
    while(i<len(passenger)):
        if(estate[2]==passenger[i][1]):
            passenger_drop=passenger.pop(i)
            sum_passenger+=1
            sum_waiting+=passenger_drop[2]+OPEN_TIME
            state[0]-=1
            i-=1
        i+=1
        
    #목적지를 결정한다.
    go_up=False
    go_down=False
    pause=False
    temp=[]
    for i in range(0, len(passenger)):
        if(temp.count(passenger[i][1])==0):
            temp.append(passenger[i][1])
                
    for i in range(0, len(call_up)):
        if(call_up[i]>state[2]):
            go_up=True
        elif(call_up[i]<state[2]):
            go_down=True
        elif(call_up[i]==state[2]):
            pause=True
            
    for i in range(0, len(call_down)):
        if(call_down[i]>state[2]):
            go_up=True
        elif(call_down[i]<state[2]):
            go_down=True
        elif(call_down[i]==state[2]):
            puase=True
            
    for i in range(0, len(temp)):
        if(temp[i]>state[2]):
            go_up=True
        elif(temp[i]<state[2]):
            go_down=True
        elif(temp[i]==state[2]):
            pause=True      
            print("오류")

    if(pause==True):
        if(state[1]==1):
            temp=[]
            for i in range(0, len(passenger_up[state[2]])):
                if((temp.count(passenger_up[state[2]][i][1])==0)and(state[2]<passenger_up[state[2]][i][1])):
                    temp.append(passenger_up[state[2]][i][1])
            for i in range(0, len(passenger_down[state[2]])):
                if((temp.count(passenger_down[state[2]][i][1])==0)and(state[2]<passenger_down[state[2]][i][1])):
                    temp.append(passenger_down[state[2]][i][1])
            for i in range(0, len(passenger)):
                if((temp.count(passenger[i][1])==0)and(state[2]<passenger[i][1])):
                    temp.append(passenger[i][1]) 
            state[3]=min(temp)
        elif(state[1]==-1):
            temp=[]
            for i in range(0, len(passenger_up[state[2]])):
                if((temp.count(passenger_up[state[2]][i][1])==0)and(state[2]>passenger_up[state[2]][i][1])):
                    temp.append(passenger_up[state[2]][i][1])
            for i in range(0, len(passenger_down[state[2]])):
                if((temp.count(passenger_down[state[2]][i][1])==0)and(state[2]>passenger_down[state[2]][i][1])):
                    temp.append(passenger_down[state[2]][i][1])
            for i in range(0, len(passenger)):
                if((temp.count(passenger[i][1])==0)and(state[2]>passenger[i][1])):
                    temp.append(passenger[i][1])
            state[3]=max(temp)
        elif(state[1]==0):
            temp=[]     
            for i in range(0, len(passenger_up[state[2]])):
                if(temp.count(passenger_up[state[2]][i][1])==0):
                    temp.append(passenger_up[state[2]][i][1])
            for i in range(0, len(passenger_down[state[2]])):
                if(temp.count(passenger_down[state[2]][i][1])==0):
                    temp.append(passenger_down[state[2]][i][1])
            for i in range(0, len(passenger)):
                if(temp.count(passenger[i][1])==0):
                    temp.append(passenger[i][1])
            for i in range(1,10):
                if(temp.count(state[2]-i)!=0):
                    state[1]=-1
                    state[3]=state[2]-i
                    break
                elif(temp.count(state[2]+i)!=0):
                    state[1]=1
                    state[3]=state[2]+i
                    break
    else:
        temp=[]
        for i in range(0, len(passenger)):
            if(temp.count(passenger[i][1])==0):
                temp.append(passenger[i][1])
        if((go_up==True)and(go_down==True)):
            if(state[1]==1): # 현재층보다 높은 층에서 온 요청 중 가장 낮은 층에서 온 요청
                dest=100
                for i in range(0, len(call_up)):
                    if((state[2]<call_up[i])and(call_up[i]<dest)):
                        dest=ecall_up[i] 
                for i in range(0, len(call_down)):
                    if((state[2]<call_down[i])and(call_down[i]<dest)):
                        dest=call_down[i] 
                for i in range(0, len(temp)):
                    if((state[2]<temp[i])and(temp[i]<dest)):
                        dest=temp[i] 
                
                state[3]=dest
            elif(state[1]==-1): # 현재층보다 낮은 층에서 온 요청 중 가장 높은 층에서 온 요청
                dest=-1
                for i in range(0, len(call_up)):
                    if((state[2]>call_up[i])and(call_up[i]>dest)):
                        dest=call_up[i] 
                for i in range(0, len(call_down)):
                    if((state[2]>call_down[i])and(call_down[i]>dest)):
                        dest=call_down[i] 
                for i in range(0, len(temp)):
                    if((state[2]>temp[i])and(temp[i]>dest)):
                        dest=temp[i] 
                state[3]=dest
            elif(state[1]==0):
                for i in range(1,10):
                    if(call_up.count(state[2]+i)!=0):
                        state[1]=1
                        state[3]=state[2]+i
                        break
                    if(call_down.count(state[2]+i)!=0):
                        state[1]=1
                        state[3]=state[2]+i
                        break
                    if(temp.count(state[2]+i)!=0):
                        state[1]=1
                        state[3]=state[2]+i
                        break
                    if(call_up.count(state[2]-i)!=0):
                        state[1]=-1
                        state[3]=state[2]-i
                        break
                    if(call_down.count(state[2]-i)!=0):
                        state[1]=-1
                        state[3]=state[2]-i
                        break
                    if(temp.count(state[2]-i)!=0):
                        state[1]=-1
                        state[3]=state[2]-i
                        break
        elif((go_up==True)and(go_down==False)):
            for i in range(1,10):
                if(call_up.count(state[2]+i)!=0):
                    state[1]=1
                    state[3]=state[2]+i
                    break
                if(call_down.count(state[2]+i)!=0):
                    state[1]=1
                    state[3]=state[2]+i
                    break
                if(temp.count(state[2]+i)!=0):
                    state[1]=1
                    state[3]=state[2]+i
                    break
        elif((go_up==False)and(go_down==True)):
            for i in range(1,10):
                if(call_up.count(state[2]-i)!=0):
                    state[1]=-1
                    state[3]=state[2]-i
                    break
                if(call_down.count(state[2]-i)!=0):
                    state[1]=-1
                    state[3]=state[2]-i
                    break
                if(temp.count(state[2]-i)!=0):
                    state[1]=-1
                    state[3]=state[2]-i
                    break
        elif((go_up==False)and(go_down==False)):
            state[1]=0
            state[3]=state[2]
    
    if(state[1]!=0):
        state[4]=now+OPEN_TIME+CLOSE_TIME+moveTime(state[2],state[3])
        sum_distance+=abs(state[3]-state[2])

    #승객을 태운다.
    while(True):
        if((len(passenger_up[state[2]])==0)and(state[1]==1)):
            break
        elif((len(passenger_down[state[2]])==0)and(state[1]==-1)):
            break
        elif(state[1]==0):
            break
        elif(state[1]==1):
            call_up.remove(state[2])
            new_passenger=passenger_up[state[2]].pop(0)
            passenger.append(new_passenger)
            state[0]+=1
            if(state[0]==LIMIT):
                break
        elif(state[1]==-1):
            call_down.remove(state[2])
            new_passenger=passenger_down[state[2]].pop(0)
            passenger.append(new_passenger)
            state[0]+=1
            if(state[0]==LIMIT):
                break
    
# 전층 엘리베이터의 n층에서 m층까지의 이동시간을 구하는 함수
def moveTime(n,m):
    return (5+2*(abs(n-m)))

       
# 2. 엘리베이터에 승객을 태우고 내리는 시뮬레이션을 진행하고, 성능을 평가한다.

#일주일 동안 엘리베이터를 운행한다
#월요일 : 0~86399초, 화요일 86400~172799초,... 일요일 518400~604800초
for i in range(0, 604800): # 1-> 604800  
    # 푸아송 분포를 통해 1초마다 탑승을 희망하는 승객을 생성한다.
    for j in range(0, 10):
        x=np.random.poisson(time_seat2[j][i]*CONGESTION, 1)
        new_passenger=x[0]
        for k in range(0, new_passenger):
            arrival=randArrival(i)
            if(arrival>j): #출발 층보다 도착 층이 높으면 up 큐에 삽입
                passenger_up[j].append([j,arrival,0])
                call_up.append(j)
            if(arrival<j): #출발 층보다 도착 층이 낮으면 down 큐에 삽입
                passenger_down[j].append([j,arrival,0])
                call_down.append(j)
                    
    
    #도착시간이 되면 엘리베이터를 멈춘다.
    if(state[4]<=i):
        state[1]=0
    #엘리베이터가 멈춰있다면 승객을 내리고, 목적지를 정하고, 승객을 태운다.
    if(state[1]==0):
        move1(i)
        #e1_move2()
        #e1_move3()
        
    
    
    # 승객의 대기시간을 1초 증가시킨다.
    for j in range(0, 10):
        for k in range(0, len(passenger_up[j])):
            passenger_up[j][k][2]+=1
        for k in range(0, len(passenger_down[j])):
            passenger_down[j][k][2]+=1
    for j in range(0, len(passenger)):
        passenger[j][2]+=1
    
    if(i%10000==0):
        print(str(i))
    
    
    
    

print("엘리베이터 ")
for i in range(0, 10):
    print(passenger_up[i])
    print(passenger_down[i])
print("직행 엘리베이터 ")
for i in range(0, 2):
    print(passenger1to9[i])

print("------------------")
print(call_up)
print(call_down)
print(state)
print("------------------")
print()
print()
print()
average_distance=sum_distance/sum_passenger
average_waiting=sum_waiting/sum_passenger
print("일주일 간 엘리베이터가 이동한 거리 : "+str(sum_distance))
print("일주일 간 엘리베이터를 사용한 승객의 수 : "+str(sum_passenger))
print("엘리베이터의 이동 거리 / 총 승객 수: "+str(average_distance))
print("승객들의 평균 이동시간"+str(average_waiting))
print("이동하는데 가장 오래걸린 승객 : "+str(std_waiting))



C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
전층 엘리베이터 
[[0, 9, 492], [0, 6, 460], [0, 8, 452], [0, 9, 387], [0, 8, 335], [0, 8, 312], [0, 8, 274], [0, 1, 266], [0, 7, 259], [0, 8, 244], [0, 8, 235], [0, 2, 234], [0, 4, 212], [0, 8, 187], [0, 8, 125], [0, 9, 81]]
[]
[[1, 8, 603], [1, 9, 206]]
[[1, 0, 449], [1, 0, 235]]
[[2, 4, 674], [2, 8, 363], [2, 8, 348], [2, 8, 345], [2, 8, 204], [2, 6, 195], [2, 3, 149]]
[]
[[3, 8, 2647], [3, 6, 1630], [3, 6, 1251], [3, 8, 1176], [3, 6, 1139], [3, 9, 1065], [3, 6, 1065], [3, 6, 504], [3, 8, 415]]
[]
[[4, 8, 1702], [4, 8, 1668], [4, 6, 975], [4, 7, 884], [4, 8, 546], [4, 6, 448]]
[]
[]
[]